# Valores faltantes

Los valores faltantes suceden. Esté preparado para este desafío común en conjuntos de datos reales.

En este tutorial, aprenderá tres enfoques para lidiar con valores faltantes. Luego comparará la efectividad de estos enfoques en un conjunto de datos del mundo real.

### Introducción

Hay muchas formas en que los datos pueden terminar con valores faltantes. Por ejemplo,

=> Una casa de 2 dormitorios no incluirá el valor del tamaño de un tercer dormitorio.

=> Un encuestado puede optar por no compartir sus ingresos.

La mayoría de las bibliotecas de aprendizaje automático (incluida scikit-learn) dan un error si intentas construir un modelo usando datos con valores faltantes. Por lo tanto, deberá elegir una de las estrategias siguientes.

### Tres enfoques

1) Una opción simple: eliminar columnas con valores faltantes

La opción más sencilla es eliminar las columnas con valores faltantes.

A menos que falten la mayoría de los valores en las columnas eliminadas, el modelo pierde acceso a mucha información (¡potencialmente útil!) con este enfoque. Como ejemplo extremo, considere un conjunto de datos con 10.000 filas, donde a una columna importante le falta una sola entrada. ¡Este enfoque eliminaría la columna por completo!

2) Una mejor opción: la imputación

La imputación completa los valores faltantes con algún número. Por ejemplo, podemos completar el valor medio en cada columna.

El valor imputado no será exactamente correcto en la mayoría de los casos, pero generalmente conduce a modelos más precisos que los que se obtendrían si eliminara la columna por completo.

3) Una extensión de la imputación

La imputación es el enfoque estándar y normalmente funciona bien. Sin embargo, los valores imputados pueden estar sistemáticamente por encima o por debajo de sus valores reales (que no se recopilaron en el conjunto de datos). O las filas con valores faltantes pueden ser únicas de alguna otra manera. En ese caso, su modelo haría mejores predicciones al considerar qué valores faltaban originalmente.

En este enfoque, imputamos los valores faltantes, como antes. Y, además, para cada columna a la que le faltan entradas en el conjunto de datos original, agregamos una nueva columna que muestra la ubicación de las entradas imputadas.

En algunos casos, esto mejorará significativamente los resultados. En otros casos, no ayuda en absoluto.

### Ejemplo

En el ejemplo, trabajaremos con el conjunto de datos de Melbourne Housing. Nuestro modelo utilizará información como la cantidad de habitaciones y el tamaño del terreno para predecir el precio de la vivienda.

No nos centraremos en el paso de carga de datos. En cambio, puedes imaginar que estás en un punto en el que ya tienes los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

### Definir función para medir la calidad de cada enfoque

Definimos una función score_dataset() para comparar diferentes enfoques para tratar con valores faltantes. Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Puntuación del método 1 (eliminar columnas con valores faltantes)

Dado que estamos trabajando con conjuntos de entrenamiento y validación, tenemos cuidado de eliminar las mismas columnas en ambos DataFrames.

In [4]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


### Puntuación del Método 2 (Imputación)

A continuación, usamos SimpleImputer para reemplazar los valores faltantes con el valor medio en cada columna.

Aunque es simple, completar el valor medio generalmente funciona bastante bien (pero esto varía según el conjunto de datos). Si bien los estadísticos han experimentado con formas más complejas de determinar los valores imputados (como la imputación de regresión, por ejemplo), las estrategias complejas generalmente no brindan ningún beneficio adicional una vez que se conectan los resultados a modelos sofisticados de aprendizaje automático.

In [5]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


Vemos que el Método 2 tiene un MAE más bajo que el Método 1, por lo que el Método 2 funcionó mejor en este conjunto de datos.

### Puntuación del método 3 (una extensión de la imputación)

A continuación, imputamos los valores faltantes y al mismo tiempo realizamos un seguimiento de qué valores se imputaron.

In [6]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


Como podemos ver, el Método 3 tuvo un desempeño ligeramente peor que el Método 2.

### Entonces, ¿por qué la imputación funcionó mejor que eliminar las columnas?

Los datos de entrenamiento tienen 10864 filas y 12 columnas, donde tres columnas contienen datos faltantes. Para cada columna, faltan menos de la mitad de las entradas. Por lo tanto, eliminar las columnas elimina mucha información útil, por lo que tiene sentido que la imputación funcione mejor.

In [7]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


### Conclusión

Como es común, imputar valores faltantes (en los Métodos 2 y 3) arrojó mejores resultados, en comparación con cuando simplemente eliminamos las columnas con valores faltantes (en el Método 1).

# EJERCICIOS

En este ejercicio, trabajará con datos del concurso de precios de vivienda para usuarios de Kaggle Learn.

Ejecute la siguiente celda de código sin cambios para cargar los conjuntos de entrenamiento y validación en X_train, X_valid, y_train e y_valid. El conjunto de prueba se carga en X_test.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Utilice la siguiente celda de código para imprimir las primeras cinco filas de datos.

In [ ]:
X_train.head()

Ya puedes ver algunos valores faltantes en las primeras filas. En el siguiente paso, obtendrá una comprensión más completa de los valores faltantes en el conjunto de datos.

### Paso 1: investigación preliminar

Ejecute la celda de código a continuación sin cambios.

In [ ]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

    (1168, 36)
    LotFrontage    212
    MasVnrArea       6
    GarageYrBlt     58
    dtype: int64

### Parte A

Utilice el resultado anterior para responder las siguientes preguntas.

In [ ]:
# Fill in the line below: How many rows are in the training data?
num_rows = 1168

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = 3 

# Fill in the line below: How many missing entries are contained in 
# all of the training data?
tot_missing = 276

### Parte B

Teniendo en cuenta sus respuestas anteriores, ¿cuál cree que es probablemente el mejor enfoque para abordar los valores faltantes?

Dado que faltan relativamente pocas entradas en los datos (a la columna con el mayor porcentaje de valores faltantes le faltan menos del 20% de sus entradas), podemos esperar que eliminar columnas probablemente no produzca buenos resultados. Esto se debe a que estaríamos desperdiciando muchos datos valiosos y, por lo tanto, la imputación probablemente funcionará mejor.

Para comparar diferentes enfoques para tratar los valores faltantes, usará la misma función score_dataset() del tutorial. Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Paso 2: elimine las columnas con valores faltantes

En este paso, procesará previamente los datos en X_train y X_valid para eliminar columnas con valores faltantes. Establezca los DataFrames preprocesados en reduce_X_train y reduce_X_valid, respectivamente.

In [ ]:
# Fill in the line below: get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()] 
# Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

Ejecute la siguiente celda de código sin cambios para obtener el MAE para este enfoque.

In [ ]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

    MAE (Drop columns with missing values):
    17837.82570776256

### Paso 3: Imputación
#### Parte A

Utilice la siguiente celda de código para imputar los valores faltantes con el valor medio a lo largo de cada columna. Establezca los DataFrames preprocesados en imputed_X_train e imputed_X_valid. Asegúrese de que los nombres de las columnas coincidan con los de X_train y X_valid.

In [ ]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer() 
# Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

Ejecute la siguiente celda de código sin cambios para obtener el MAE para este enfoque.

In [ ]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

    MAE (Imputation):
    18062.894611872147

### Parte B

Compare el MAE de cada enfoque. ¿Te sorprende algo de los resultados? ¿Por qué cree que un enfoque funcionó mejor que el otro?

Dado que hay tan pocos valores faltantes en el conjunto de datos, esperaríamos que la imputación funcionara mejor que eliminar columnas por completo. Sin embargo, vemos que eliminar columnas funciona ligeramente mejor. Si bien esto probablemente pueda atribuirse en parte al ruido en el conjunto de datos, otra posible explicación es que el método de imputación no coincide muy bien con este conjunto de datos. Es decir, tal vez en lugar de completar el valor medio, tenga más sentido establecer cada valor faltante en un valor de 0, completar el valor encontrado con más frecuencia o utilizar algún otro método. Por ejemplo, considere la columna GarageYrBlt (que indica el año en que se construyó el garaje). Es probable que, en algunos casos, un valor faltante indique una casa que no tiene garaje. ¿Tiene más sentido completar el valor mediano en cada columna en este caso? ¿O podríamos obtener mejores resultados completando el valor mínimo en cada columna? No está del todo claro qué es lo mejor en este caso, pero tal vez podamos descartar algunas opciones de inmediato; por ejemplo, establecer los valores faltantes en esta columna en 0 probablemente produzca resultados horribles.

### Paso 4: generar predicciones de prueba

En este paso final, utilizará cualquier enfoque que elija para abordar los valores faltantes. Una vez que haya preprocesado las funciones de entrenamiento y validación, entrenará y evaluará un modelo de bosque aleatorio. Luego, procesará previamente los datos de la prueba antes de generar predicciones que puedan enviarse a la competencia.

#### Parte A

Utilice la siguiente celda de código para preprocesar los datos de entrenamiento y validación. Establezca los DataFrames preprocesados en final_X_train y final_X_valid. ¡Puedes utilizar cualquier enfoque que elijas aquí! Para que este paso se marque como correcto solo debes asegurarte de:

. los DataFrames preprocesados tienen el mismo número de columnas,

. los DataFrames preprocesados no tienen valores faltantes,

. final_X_train e y_train tienen el mismo número de filas, y

. final_X_valid e y_valid tienen el mismo número de filas.

In [ ]:
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Preprocessed training and validation features
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

Ejecute la siguiente celda de código para entrenar y evaluar un modelo de bosque aleatorio. (Tenga en cuenta que no usamos la función score_dataset() anterior, ¡porque pronto usaremos el modelo entrenado para generar predicciones de prueba!)

In [ ]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

    MAE (Your approach):
    17791.59899543379

### Parte B

Utilice la siguiente celda de código para preprocesar los datos de su prueba. Asegúrese de utilizar un método que coincida con la forma en que procesó previamente los datos de entrenamiento y validación, y configure las funciones de prueba preprocesadas en final_X_test.

Luego, utilice las funciones de prueba preprocesadas y el modelo entrenado para generar predicciones de prueba en preds_test.

Para que este paso se marque como correcto, solo necesita asegurarse de:

. el DataFrame de prueba preprocesado no tiene valores faltantes, y

. final_X_test tiene el mismo número de filas que X_test.

In [ ]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

Ejecute la siguiente celda de código sin cambios para guardar sus resultados en un archivo CSV que pueda enviarse directamente a la competencia.

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

### Envía tus resultados

Una vez que haya completado con éxito el Paso 4, estará listo para enviar sus resultados a la tabla de clasificación. (También aprendió cómo hacer esto en el ejercicio anterior. Si necesita un recordatorio de cómo hacerlo, siga las instrucciones a continuación).

Primero, deberás unirte a la competencia si aún no lo has hecho. Así que abre una nueva ventana haciendo clic en este enlace. Luego haga clic en el botón Unirse a la competencia.

A continuación, siga las instrucciones a continuación:

1. Comience haciendo clic en el botón Guardar versión en la esquina superior derecha de la ventana. Esto generará una ventana emergente.

2. Asegúrese de que la opción Guardar y ejecutar todo esté seleccionada y luego haga clic en el botón Guardar.

3. Esto genera una ventana en la esquina inferior izquierda del cuaderno. Una vez que haya terminado de ejecutarse, haga clic en el número a la derecha del botón Guardar versión. Esto muestra una lista de versiones a la derecha de la pantalla. Haga clic en los puntos suspensivos (...) a la derecha de la versión más reciente y seleccione Abrir en el Visor. Esto lo lleva al modo de visualización de la misma página. Deberá desplazarse hacia abajo para volver a estas instrucciones.

4. Haga clic en la pestaña Datos cerca de la parte superior de la pantalla. Luego, haga clic en el archivo que desea enviar y haga clic en el botón Enviar para enviar sus resultados a la tabla de clasificación.

¡Ya te has presentado exitosamente al concurso!

Si desea seguir trabajando para mejorar su rendimiento, seleccione el botón Editar en la parte superior derecha de la pantalla. Luego puedes cambiar tu código y repetir el proceso. Hay mucho margen de mejora y ascenderás en la clasificación a medida que trabajes.